# Running a ReCom Chain

## Set Up the chain

@author eveomett <br>
AI for Redistricting <br>
For this to run, you need PA.json and also the PA shapefile data.

In [1]:
from gerrychain.random import random
random.seed(12345678)

In [2]:
import matplotlib.pyplot as plt
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from functools import partial
import pandas as pd
import geopandas as gpd
import csv
import os
import datetime  #For keeping track of runtime
import tqdm # for progress bar

Adding time information can help you know how long you might need to run a longer chain.  It's not so useful for a jupyter notebook, but you may want to use it in your project.

In [3]:
beginrun = datetime.datetime.now()
print ("\nBegin date and time : ", beginrun.strftime("%Y-%m-%d %H:%M:%S"))


Begin date and time :  2024-04-12 16:06:42


We can create files for later analyses, if desired. <br>
I'll create files and put them in this directory:

In [4]:
outdir="./NC_recom_2020_CD/"

Here are some parameters that we can change later if we'd like. <br>
```save_district_plot_mod``` determines which plots (maps) we'll save.

In [5]:
total_steps_in_run=100
save_district_graph_mod=1
save_district_plot_mod=10

In [6]:
os.makedirs(outdir, exist_ok=True)
graph = Graph.from_file("./NC/NC.shp")

/Users/weiqianru/anaconda3/envs/gerry/lib/python3.11/site-packages/gerrychain/graph/graph.py:179: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


These are the elections we have available:

In [7]:
elections = [
    Election("G20PRE", {"Democratic": "G20PRED", "Republican": "G20PRER"}),
    Election("G20USS", {"Democratic": "G20USSD", "Republican": "G20USSR"}),
]

Remember the population updater is for computing how close to equality the district populations are. <br>
"TOT_POP" is the population column from our shapefile.

In [8]:
my_updaters = {"population": updaters.Tally("TOTPOP", alias="population")}

Election updaters, for computing election results using the vote totals from our shapefile.

In [9]:
election_updaters = {election.name: election for election in elections}
my_updaters.update(election_updaters)

This assignment is using the PA map with the Goofy kicking Donald Duck district in it.

In [10]:
initial_partition = GeographicPartition(graph, 
                                        assignment= "CD", 
                                        updaters=my_updaters)

KeyError: 'CD'

The ReCom proposal needs to know the ideal population for the districts so that we can improve speed by bailing early on unbalanced partitions.

In [ ]:
initial_partition.plot()

In [ ]:
ideal_population = sum(initial_partition["population"].values()) / len(initial_partition)

In [ ]:
proposal = partial(recom,
                   pop_col="TOTPOP",
                   pop_target=ideal_population,
                   epsilon=0.02,
                   node_repeats=2
                  )

In [ ]:
compactness_bound = constraints.UpperBound(
    lambda p: len(p["cut_edges"]),
    2*len(initial_partition["cut_edges"])
)

In [ ]:
pop_constraint = constraints.within_percent_of_ideal_population(initial_partition, 0.02)

In [ ]:
chain = MarkovChain(
    proposal=proposal,
    constraints=[
        pop_constraint,
        compactness_bound
    ],
    accept=accept.always_accept,
    initial_state=initial_partition,
    total_steps=total_steps_in_run
    )

## Signature of Gerrymandering

We will use the dataframe below to get `signature of gerrymandering'<br>
Much of this was taken from [https://gerrychain.readthedocs.io/en/latest/user/recom/#a-real-world-example](https://gerrychain.readthedocs.io/en/latest/user/recom/#a-real-world-example)

In [ ]:
data = pd.DataFrame(
    sorted(partition["G20PRE"].percents("Democratic"))
    for partition in chain.with_progress_bar()
)

In [ ]:
fig, ax = plt.subplots(figsize=(8, 6))

# Draw 50% line
ax.axhline(0.5, color="#cccccc")

# Draw boxplot
data.boxplot(ax=ax, positions=range(len(data.columns)))

# Draw initial plan's Democratic vote %s (.iloc[0] gives the first row, which corresponds to the initial plan)
plt.plot(data.iloc[0], "ro")

# Annotate
ax.set_title("Comparing the 2020 plan to an ensemble")
ax.set_ylabel("Democratic vote % (President 2020)")
ax.set_xlabel("Sorted districts")
ax.set_ylim(0, 1)
ax.set_yticks([0, 0.25, 0.5, 0.75, 1])

plt.show()

## Creating output files

Here we'll create some output files for each of our maps. <br>
For each graph, we have one csv with the partisan (election outcome) data, and one csv with the district dual graph data. <br>
We're also outputting the map every 100 steps.

In [ ]:
df=gpd.read_file("./NC/NC.shp")

In [ ]:
index_output_file = outdir + "_index.csv"

In [ ]:
with open(index_output_file, mode='w') as index_file:
    index_writer = csv.writer(index_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    
    #Run through chain, building 
    for t, part in enumerate(chain):
        if t%save_district_graph_mod==0:
            tmp_str=str(t)
            file_prefix=tmp_str.rjust(8,'0')
            
            #Add file names to index file            
            edges_output_file = outdir + file_prefix + "_edges.csv"
            districts_output_file = outdir + file_prefix + "_districts.csv"            
            index_writer.writerow(["./" + file_prefix + "_districts.csv", "./" + file_prefix + "_edges.csv"])
                        

            #Build the district graph and write edges to file    
            s=set()
            for e in part["cut_edges"]:
                s.add( (part.assignment[e[0]],part.assignment[e[1]] ))
                #s.add( (e[0],e[1]) )
                
            with open(edges_output_file, mode='w') as edges_file:
                edges_writer = csv.writer(edges_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
                for e1 in s:
                    edges_writer.writerow([e1[0],e1[1]])
               
            
            #Build the district file that contains election results
            D_votes = part["G20PRE"].votes("Democratic") 
            R_votes = part["G20PRE"].votes("Republican") 
           
            with open(districts_output_file, mode='w') as districts_file:            
                districts_writer = csv.writer(districts_file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
        
                for i in range(len(D_votes)):
                    districts_writer.writerow([i+1, int(D_votes[i]),int(R_votes[i]),0])
             
        if t%save_district_plot_mod==0:
            df.plot(pd.Series([part.assignment[i] for i in df.index]), cmap="tab20", figsize=(16,8))
            #plt.show()
            plot_output_file = outdir + file_prefix + "_plot.png"
            plt.savefig(plot_output_file)
            plt.close()
    
            

And if you want to find the total time:

In [ ]:
endrun = datetime.datetime.now()
print ("\nEnd date and time : ", endrun.strftime("%Y-%m-%d %H:%M:%S"))

    
diff=endrun-beginrun
print("\nTotal time: ", str(diff))

https://s3.us-west-1.amazonaws.com/wakegov.com.if-us-west-1/documents/2021-04/Precinct_01-07A.pdf
